# Optimal Advertising Strategy

A company is considering whether to make an ad buy for one million dollars.  In a good year, the company has sales of 4 million dollars, in a normal year, its sales are 2 million dollars, and in a bad year, its sales are only 1 million dollars.  The company thinks has an internal rate of return on other projects at gross rate 1+r

We use matrices to represent the impact of advertising; the entry in row i, column j, represents the probability of entering state j when one is currently in state i. Here, we let the first row represents a bad year, the second a normal year, and the third a good year. The columns are likewise. This would imply that the top right entry is the probability of having a bad year next year, when one has just had a bad year. 

If there is no ad buy, the Markov matrix describing transitions between states is:

$$
M(0) = \left[\begin{array}{ccc}
0.8 & 0.2 & 0 \\
0.2 & 0.8 & 0 \\
0.1 & 0.4 & 0.5 \\
\end{array} \right]
$$

If there is an ad buy, the Markov matrix describing transitions between states is:

$$
M(1) = \left[\begin{array}{ccc}
0.5 & 0.5 & 0 \\
0.1 & 0.6 & 0.3 \\
0.1 & 0.3 & 0.6 \\
\end{array} \right]
$$

As we can see, advertising significantly improves the expectation of next years sales in all three states. 

# Bellman Equations

We use Bellman equations to evaluate the firm's optimal strategy in the dynamic program. A dynamic program is a set of uncertain outcomes, strategies, and rewards. Our solution will be a set of optimal strategies for all states. For example, in this case our solution could be that the firm should always invest in advertising in good or normal years, and cut the advertising budget in bad years. 

The dynamic program is defined as follows;

1. The state space is $X = \{Bad, Normal, Good\}$
2. The control set is $U = \{0, 1\}$, where 0 means no ad campaign and 1 means make an ad buy.
3. The reward function $r(x,u)$ is given by these six numbers:

$$
r(Bad, 0) = 1 \\
r(Bad, 1) = 0 \\
r(Normal, 0) = 2 \\
r(Normal, 1) = 1 \\
r(Good, 0) = 4 \\
r(Good, 1) = 3\\
$$

4. The transition rule is described by the two Markov matrices $M(0)$ and $M(1)$.
5. The discount factor for this problem is $\beta = 1/(1+r)$

Which leads to the following Bellman equation, whose value we will seek to maximize;

$$
V(Normal) = max \left\{ 2 + \beta [0.2 V(Bad) + 0.8 V(Normal)], 1 + \beta [0.1 V(Bad) + 0.6 V(Normal)+ 0.3 V(Good)]\right\}
$$

The Bellman equation represents the certain reward for the decision in the current state, plus the expected value of the decision in the future state

We define our paramaters below

In [1]:
import numpy as np
import pandas as pd
import quantecon as qe

States=['Bad', 'Normal', 'Good']
Controls=['No buy','Buy']
reward=np.array([[1,0],[2,1],[4,3]])
beta=0.9
value=np.zeros(len(States))

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
M0=np.array([[0.8,0.2,0],[0.2,0.8,0],[0.1,0.4,0.5]])
M1=np.array([[0.5,0.5,0],[0.1,0.6,0.3],[0.1,0.3,0.6]])
markov_matrices=[M0,M1]
transition={} #empty dictionary
for control in Controls:
    transition[control]=markov_matrices[Controls.index(control)]

In [3]:
def check_parameters(States,Controls,reward,transition,value,beta):
    """
    The value function has n elements
    The control set has m elements
    The reward function is an n x m matrix
    The transition is a list of m Markov matrices, each with dimesnion n x n
    The discount factor must satisfy 0 <= beta < 1
    """
    n=len(States)
    m=len(Controls)
    if len(value) != n: print("The value function and the state space are not conformable.")
    if reward.shape != (n,m): print("The reward function is not well defined.")
    if len(transition) != m: 
            print("There is not the requisite number of transition matrices.")
    if not all(transition[control].shape == (n,n) for control in Controls):
        print("At least one of the Markov transitions is of the wrong dimension.")
    if not all(np.ones(n)@transition[control]@np.ones(n) == n for control in Controls):
        print("Give me a break! One of the transition matrices is not Markov.")
    if not all((transition[control]>=0).all() and (transition[control]<=1).all() for control in Controls):
            print("Give me a break! The elements of the transition matrices are not probabilities.")   
    if beta < 0  or beta >= 1:
        print("Something is wrong with the discount factor.")
    return (print("I have done the requisite checks."))

In [4]:
def next_state_probs(state,control,transition):
    """
    This function takes the state set, the control set, and the list of transition matrices
    It returns the relevant row of the transition matrix in state s, when control u is chosen
    """
    return(transition[control][States.index(state)])

# The Howard Method

To discover the optimal set of strategies, we use the the Howard Method. The algorithm has the following steps;

1. Make a guess for the optimal set of strategies

2. See if these strategies maximize the reward function using the Bellman Equation

3. If they do, then the solution is found, if else, repeat with the set of strategies that was higher.

We define this algorithm below

In [5]:
def next_value(States, Controls, reward, transition, value, beta):
    """
    The value function has n elements
    The control set has m elements
    The reward function is an n x m matrix
    The transition is a list of m Markov matrices, each with dimesnion n x n
    The discount factor must satisfy 0 <= beta < 1
    """
    v_next=[]
    for state in States:
        bellman=[]
        for control in Controls:
            bellman.append(reward[States.index(state),
        Controls.index(control)]+beta*(value@next_state_probs(state,control,transition)))
        v_next.append(max(bellman))
    return(v_next)

In [6]:
X=[1.1,1.2,-3,5]
X.index(min(X))

2

In [7]:
def solve(States, Controls, reward, transition, value, beta):
    check=1.0 #check for convergence
    epsilon=10e-5 #epsilon to define convergence
    max_iter = 100 #maximum iterations for the while loop
    iter=0 #count iterations
    while check > epsilon:
        v1 = np.array(next_value(States,Controls,reward,transition,value,beta))
        check=max(abs(value-v1))
        value = v1 #reset the value function
        iter += 1 #advance the counter
        if(iter > max_iter): #get out of Dodge, if we are in an infinite loop
            print("Algorithm did not converge")
            break
    return(value) 

value=solve(States,Controls,reward,transition,value,beta)

In [8]:
def optimal_policy(States,Controls,reward,transition,value,beta):
    policy=[]
    for state in States:
        z=[]
        for control in Controls:
            z.append(reward[States.index(state),Controls.index(control)]+beta*(value@next_state_probs(state,control,transition)))
        policy.append(Controls[z.index(max(z))])
    return(policy)

policy=optimal_policy(States,Controls,reward,transition,value,beta)

In [9]:
def M_optimal(States,transition,policy):
    M=np.empty_like(list(transition.items())[0][1])
    for state in States:
        M[States.index(state),:]=transition[policy[States.index(state)]][States.index(state),:]
    return(M)

P=M_optimal(States,transition,policy)
from quantecon import MarkovChain
mc = qe.MarkovChain(P)
p_stationary=mc.stationary_distributions  # Show all stationary distributions

In [10]:
PS5=pd.DataFrame(reward, States, Controls)
PS5["value function"]=value
PS5["optimal policy"]=policy
PS5["probabilities"]=p_stationary[0] #only one distribution in the list that was returned
PS5

,No buy,Buy,value function,optimal policy,probabilities
Bad,1,0,14.804697,No buy,0.333333
Normal,2,1,17.474440,Buy,0.416667
Good,4,3,21.132976,No buy,0.250000
